# 保存和加载检查点

在本教程中，我们将探讨如何使用`orbax`库以及`braintools`的轻量级方法在`brainstate`中保存和加载检查点。这对于在训练过程中保存模型状态非常有用，这样您可以从中断的地方继续训练或稍后使用已训练的模型进行推理。以下示例演示了如何将`orbax`和`braintools`的检查点功能与一个简单的多层感知机(MLP)模型结合使用。

首先，您可以通过运行以下命令安装`orbax`库：

`pip install orbax-checkpoint`

您也可以直接从 GitHub 安装，使用以下命令。这可以用来获取 Orbax 的最新版本。

`pip install 'git+https://github.com/google/orbax/#subdirectory=checkpoint'`

其次，您可以通过运行以下命令安装`braintools`库：

`pip install braintools`

首先，我们将导入所需的库：

In [18]:
import tempfile
import os

import jax
import jax.numpy as jnp
import orbax.checkpoint as orbax
import braintools

import brainstate

## 定义模型
我们使用`brainstate`来定义一个简单的多层感知机(MLP)模型。

In [19]:
class MLP(brainstate.nn.Module):
    def __init__(self, din: int, dmid: int, dout: int):
        super().__init__()
        self.dense1 = brainstate.nn.Linear(din, dmid)
        self.dense2 = brainstate.nn.Linear(dmid, dout)

    def __call__(self, x: jax.Array) -> jax.Array:
        x = self.dense1(x)
        x = jax.nn.relu(x)
        x = self.dense2(x)
        return x

## 创建模型
我们将设置随机数种子来实例化模型。

In [20]:
SEED = 42
brainstate.random.seed(SEED)   # 在brainstate中设置随机种子
model1 = MLP(10, 20, 30)    # 创建模型
model1

MLP(
  dense1=Linear(
    in_size=(10,),
    out_size=(20,),
    w_mask=None,
    weight=ParamState(
      value={'weight': Array([[ 0.74939334,  0.3148138 ,  0.60089725, -0.7131149 ,  0.6790908 ,
              -0.44663328,  0.03113358, -0.5250644 ,  0.1614144 , -0.39722365,
              -0.23442519,  0.118144  ,  0.7669531 ,  0.06876656,  0.6045511 ,
               0.12086334, -0.88447595, -0.19188431, -0.85868365,  0.00500867],
             [ 0.20412642,  0.07092498,  0.37392026,  0.34958398, -0.57214   ,
               0.71724516, -0.08160591,  0.50068825, -0.17175189, -0.08275215,
               0.6508336 ,  0.28279537,  0.08821856,  0.83949256,  0.49844882,
              -0.04159267, -0.47324428,  0.27084318, -0.58236146, -0.09787997],
             [-0.04382031, -0.20300323, -0.04449642,  0.41578326,  0.5507486 ,
              -0.15913244, -0.8612537 ,  0.19072336, -0.16082875, -0.24696219,
              -0.30372635,  0.6850187 ,  0.32007053,  0.24253711,  0.28217098,
           

## 保存模型参数

### 使用`orbax`保存检查点
我们将模型参数保存到检查点文件中。

In [21]:
tmpdir = tempfile.mkdtemp()    # 创建临时目录
state_tree = brainstate.graph.treefy_states(model1)    # 将模型的状态转换为树结构
checkpointer = orbax.PyTreeCheckpointer()   # 创建检查点对象
checkpointer.save(os.path.join(tmpdir, 'state'), state_tree)    # 保存模型的参数

现在，我们已经将模型的参数通过`orbax`保存到`tmpdir/state`的检查点文件中。

### 使用`braintools`保存检查点

In [22]:
checkpoint = brainstate.graph.states(model1).to_nest()   # 将模型的状态转换为nest结构
braintools.file.msgpack_save(os.path.join(tmpdir, 'state.msgpack'), checkpoint)    # 保存模型的参数

Saving checkpoint into C:\Users\13107\AppData\Local\Temp\tmp483fc4t1\state.msgpack


现在，我们已经将模型的参数通过`braintools`保存到`tmpdir/state.msgpack`的检查点文件中。

## 加载模型参数

### 使用`orbax`加载检查点
我们将从检查点文件中加载模型的参数。

In [23]:
# 创建一个有着相同结构的模型
brainstate.random.seed(0)
model2 = brainstate.augment.abstract_init(lambda: MLP(10, 20, 30))
state_tree = brainstate.graph.treefy_states(model2)

# 从检查点文件读取模型参数
checkpointer = orbax.PyTreeCheckpointer()
state_tree = checkpointer.restore(os.path.join(tmpdir, 'state'), item=state_tree)

# 更新模型的状态
brainstate.graph.update_states(model2, state_tree)

c:\routhleck_app\miniconda\envs\brainstate\lib\site-packages\orbax\checkpoint\_src\serialization\type_handlers.py:1123: UserWarning: Couldn't find sharding info under RestoreArgs. Populating sharding info from sharding file. Please note restoration time will be slightly increased due to reading from file instead of directly from RestoreArgs. Note also that this option is unsafe when restoring on a different topology than the checkpoint was saved with.
  warnings.warn(


### 使用`braintools`加载检查点
我们将从检查点文件中加载模型的参数。

In [24]:
# 创建一个有着相同结构的模型
brainstate.random.seed(0)
model3 = brainstate.augment.abstract_init(lambda: MLP(10, 20, 30))
checkpoint = brainstate.graph.states(model3).to_nest()

# 从msgpack文件读取模型参数
braintools.file.msgpack_load(os.path.join(tmpdir, 'state.msgpack'), checkpoint)

Loading checkpoint from C:\Users\13107\AppData\Local\Temp\tmp483fc4t1\state.msgpack


{'dense1': {'weight': ParamState(
    value={'bias': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0.], dtype=float32), 'weight': array([[ 0.74939334,  0.3148138 ,  0.60089725, -0.7131149 ,  0.6790908 ,
            -0.44663328,  0.03113358, -0.5250644 ,  0.1614144 , -0.39722365,
            -0.23442519,  0.118144  ,  0.7669531 ,  0.06876656,  0.6045511 ,
             0.12086334, -0.88447595, -0.19188431, -0.85868365,  0.00500867],
           [ 0.20412642,  0.07092498,  0.37392026,  0.34958398, -0.57214   ,
             0.71724516, -0.08160591,  0.50068825, -0.17175189, -0.08275215,
             0.6508336 ,  0.28279537,  0.08821856,  0.83949256,  0.49844882,
            -0.04159267, -0.47324428,  0.27084318, -0.58236146, -0.09787997],
           [-0.04382031, -0.20300323, -0.04449642,  0.41578326,  0.5507486 ,
            -0.15913244, -0.8612537 ,  0.19072336, -0.16082875, -0.24696219,
            -0.30372635,  0.6850187 ,  0.32007053,  0.

## 验证加载的模型
让我们运行加载的模型并检查它是否产生与原始模型相同的输出。

In [25]:
y1 = model1(jnp.ones((1, 10)))
y2 = model2(jnp.ones((1, 10)))
y3 = model3(jnp.ones((1, 10)))
print(jnp.allclose(y1, y2))    # True
print(jnp.allclose(y1, y3))    # True

True
True
